In [1]:
!nvidia-smi

Tue Oct 31 17:35:39 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.98                 Driver Version: 535.98       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti     Off | 00000000:1A:00.0 Off |                  N/A |
| 22%   23C    P8              11W / 250W |      0MiB / 11264MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
import pickle
import torch
import numpy as np
import pandas as pd   
import os
from pathlib import Path
import glob
import json

import os

import random
from transformers import AutoImageProcessor, ViTModel
import torch


/nfshomes/canum05/miniconda3/envs/llm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [46]:
#train_dataframe and val_dataframe stores the path to the images and respective questions and answers
lenghtData = 2000

trainList=[]
with open('/nfshomes/canum05/scratch/dataset/CLEVR_v1.0/questions/CLEVR_train_questions.json') as f:
    data = json.load(f)
    for K in range(lenghtData):
        i = data['questions'][random.randrange(20, 2000, 3)]
        temp=[]
        for path in glob.glob('/nfshomes/canum05/scratch/dataset/CLEVR_v1.0/images/train/'+i['image_filename']): 
            temp.append(path)
        temp.append(i['question'])
        temp.append(i['answer'])
        trainList.append(temp)
f.close()
labels=['Path','Question','Answer']
train_dataframe = pd.DataFrame.from_records(trainList, columns=labels)#training Dataframe 
del(data)
del(trainList)

In [47]:
train_dataframe

,Path,Question,Answer
0,/nfshomes/canum05/scratch/dataset/CLEVR_v1.0/i...,How many other things are there of the same ma...,3
1,/nfshomes/canum05/scratch/dataset/CLEVR_v1.0/i...,The green thing that is the same size as the p...,cube
2,/nfshomes/canum05/scratch/dataset/CLEVR_v1.0/i...,Is there a large cube that is to the right of ...,yes
3,/nfshomes/canum05/scratch/dataset/CLEVR_v1.0/i...,The object that is in front of the cyan cylind...,gray
4,/nfshomes/canum05/scratch/dataset/CLEVR_v1.0/i...,How many rubber blocks have the same color as ...,1
...,...,...,...
1995,/nfshomes/canum05/scratch/dataset/CLEVR_v1.0/i...,How many objects are tiny metal objects or tin...,3
1996,/nfshomes/canum05/scratch/dataset/CLEVR_v1.0/i...,There is a tiny object in front of the tiny gr...,yes
1997,/nfshomes/canum05/scratch/dataset/CLEVR_v1.0/i...,What is the size of the metal sphere that is r...,large
1998,/nfshomes/canum05/scratch/dataset/CLEVR_v1.0/i...,The green thing that is the same size as the p...,cube


In [12]:
image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")
model = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k")


In [54]:
from PIL import Image
from tqdm import tqdm

In [55]:

data_list = []

for index, row in tqdm(train_dataframe.iterrows(), total=len(train_dataframe)):
    path = row['Path']
    question = row['Question']
    answer = row['Answer']

    # Open and load the image using Pillow
    img = Image.open(path).convert("RGB")
    inputs = image_processor(img, return_tensors="pt")

    with torch.no_grad():
        outputs = model(**inputs)

    embeddings = outputs.last_hidden_state.tolist()  # Convert embeddings to a list

    sample_data = {
        'image_embeddings': embeddings,
        'question': question,
        'answer': answer
    }

    data_list.append(sample_data)


with open('./Data/data.json', 'w') as json_file:
    json.dump(data_list, json_file)




100%|██████████| 2000/2000 [23:56<00:00,  1.39it/s]


In [3]:
json_file_path = './Data/data.json'

# Read the JSON file into a Pandas DataFrame
df = pd.read_json(json_file_path)

In [4]:
df

,image_embeddings,question,answer
0,"[[[0.359248578548431, -0.6500996351242061, -0....",How many other things are there of the same ma...,3
1,"[[[0.11450660973787301, -0.608290016651153, -0...",The green thing that is the same size as the p...,cube
2,"[[[0.13305327296257002, -0.573591530323028, -0...",Is there a large cube that is to the right of ...,yes
3,"[[[0.14678417146205902, -0.359456837177276, -0...",The object that is in front of the cyan cylind...,gray
4,"[[[0.361140221357345, -0.380545675754547, -0.3...",How many rubber blocks have the same color as ...,1
...,...,...,...
1995,"[[[0.11589656025171201, -0.387044250965118, -0...",How many objects are tiny metal objects or tin...,3
1996,"[[[0.34504896402359003, -0.582923293113708, 0....",There is a tiny object in front of the tiny gr...,yes
1997,"[[[0.34835848212242104, -0.357507735490798, -0...",What is the size of the metal sphere that is r...,large
1998,"[[[0.11450660973787301, -0.608290016651153, -0...",The green thing that is the same size as the p...,cube


In [12]:
emb = torch.tensor(df['image_embeddings'][0])


In [13]:
emb.shape

torch.Size([1, 197, 768])